# Get Data

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from scipy.sparse import csr_matrix

from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler

from nltk.corpus import stopwords

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import nltk
import string
import re

from collections import Counter

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

We want to gain some info about data.

In [ ]:
data = pd.read_csv("../input/spam.csv", encoding='latin-1')
data.head()

In [ ]:
data.info()

The 3 last columns seem to be useless, as they do not mostly contain a value and are null.

In [ ]:
unimportant_col = ["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"]
useful_data = data.drop(unimportant_col, axis=1)

It's better to rename the columns names for better understanding.

In [ ]:
useful_data = useful_data.rename(columns={"v1": "Type", "v2": "Text"})
useful_data.head()

# Create a Test Set

At the beginning, we choose a part of data for test and we will put it aside. So, our computation will not depend on it.

In [ ]:
train_data, test_data = train_test_split(useful_data, test_size=0.2, random_state=42)

# Analyze Data

In [ ]:
train_data.info()

In [ ]:
train_data.describe()

In [ ]:
train_data.groupby("Type").describe()

In [ ]:
train_data.Type.value_counts().plot.pie();

# Add Length Feature

Now, we will add new features and check if they are useful.

In [ ]:
train_data['Length'] = train_data['Text'].apply(len)
train_data.head()

As seen below, ham messages are usually much shorter than the spam ones.

In [ ]:
train_data.hist(column='Length',by='Type',bins=60,figsize=(12,4))

# Add UpperCase Count Feature

We now want to analyze the number of words which are fully in uppercase.

In [ ]:
findCapitalCount = lambda x: sum(map(str.isupper, x['Text'].split())) 
train_data["CapitalCount"] = train_data.apply(findCapitalCount, axis=1)
train_data.head()

This feature can be useful too.

As visualized below, spam messages are more likely to have these kind of words.

In [ ]:
train_data.hist(column='CapitalCount',by='Type',bins=30,figsize=(12,4))

# Add Word Count Feature

In [ ]:
findWordCount = lambda x: len(x['Text'].split()) 
train_data["WordCount"] = train_data.apply(findWordCount, axis=1)
train_data.head()

# Add UpperCase Rate Feature

The number of words in general or the number of uppercase words may not be accurate separately and may not lead to preceise results. 

It is better to get the ratio of the uppercase words count to all words.

In [ ]:
train_data["CapitalRate"] = train_data["CapitalCount"] / train_data["WordCount"]
train_data.head()

In [ ]:
train_data.hist(column='CapitalRate',by='Type',bins=30,figsize=(12,4))

# Common Words

Some words such as 'and', 'it', etc. are very common in english texts and can be misleading. We can delete them from our text.

In [ ]:
stopwords.words("english")

# Stemmer

It is better to analyze the root of words as they can come in different forms.

Stemmer can do the job for us!

In [ ]:
stemmer = nltk.PorterStemmer()
for word in ("Computations", "Computation", "Computing", "Computed", "Compute", "Compulsive"):
    print(word, "=>", stemmer.stem(word))

# Pipeline

We need to define some transformers, in order to create a pipeline.

In [ ]:
class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attr):
        self.attr_names = attr
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        X = X.rename(columns={"v1": "Type", "v2": "Text"})
        return X[self.attr_names]

As mentioned above, we can add useful attributes such as Length, and CapitalRate.

In [ ]:
class AddExtraAttr(BaseEstimator, TransformerMixin):
    def __init__(self, attr):
        self.attr_name = attr
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        X = X.values
        X = pd.DataFrame(X, columns=[self.attr_name])
        X['Length'] = X[self.attr_name].apply(len)
#         findCapitalCount = lambda x: sum(map(str.isupper, x[self.attr_name].split())) 
#         X["CapitalCount"] = X.apply(findCapitalCount, axis=1)
#         findWordCount = lambda x: len(x[self.attr_name].split()) 
#         X["WordCount"] = X.apply(findWordCount, axis=1)
#         X["CapitalRate"] = X["CapitalCount"] / X["WordCount"]
        return X[["Length"]]

At this step, we replace the sequence of integers with a NUMBER string. Then we seperate the text from punctuation for easier analysis. Then we need to eliminate the common english words from SMS content, as they can easily mislead us, and finally stem the words.

Now we can count the words.

In [ ]:
class FindCount(BaseEstimator, TransformerMixin):
    def __init__(self, attr):
        self.attr_name = attr
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        Y = []
        for index, row in X.iterrows():
            row[self.attr_name] = re.sub(r'\d+(?:\.\d*(?:[eE]\d+))?', 'NUMBER', row[self.attr_name])
            no_punc = [ch for ch in row[self.attr_name] if ch not in string.punctuation]
            punc = [(" " + ch + " ") for ch in row[self.attr_name] if ch in string.punctuation]
            word_list = "".join(no_punc + punc).split()
            useful_words = [word.lower() for word in word_list if word.lower() not in stopwords.words("english")]
            word_counts = Counter(useful_words)
            stemmed_word_counts = Counter()
            for word, count in word_counts.items():
                stemmed_word = stemmer.stem(word)
                stemmed_word_counts[stemmed_word] += count
            word_counts = stemmed_word_counts
            Y.append(word_counts)
        return Y

Now, we convert the count data to vector, and create a sparse matrix.

This will show us, the number of occurances of each word in each text.

In [ ]:
class ConvertToVector(BaseEstimator, TransformerMixin):
    def __init__(self, vocabulary_size=1000):
        self.vocabulary_size = vocabulary_size
    def fit(self, X, y=None):
        total_count = Counter()
        for word_count in X:
            for word, count in word_count.items():
                total_count[word] += count
        most_common = total_count.most_common()[:self.vocabulary_size]
        self.most_common_ = most_common
        self.vocabulary_ = {word: index + 1 for index, (word, count) in enumerate(most_common)}
        return self
    def transform(self, X, y=None):
        rows = []
        cols = []
        data = []
        for row, word_count in enumerate(X):
            for word, count in word_count.items():
                rows.append(row)
                cols.append(self.vocabulary_.get(word, 0))
                data.append(count)
        return csr_matrix((data, (rows, cols)), shape=(len(X), self.vocabulary_size + 1))

Using the transformers above, we can make a pipeline to automate the process.

In [ ]:
cat_attr = ["Text"]

main_pipeline = Pipeline([
        ('selector', DataFrameSelector(cat_attr)),
        ('find_count', FindCount(cat_attr[0])),
        ('convert_to_vector', ConvertToVector()),
    ])

extra_pipeline = Pipeline([
        ('selector', DataFrameSelector(cat_attr)),
        ('add_extra', AddExtraAttr(cat_attr[0])),
    ])

full_pipeline = FeatureUnion(transformer_list=[
    ('main_pipeline', main_pipeline),
    ('extra_pipeline', extra_pipeline),
])

In [ ]:
train_prepared = full_pipeline.fit_transform(train_data)

# Train Model

As the types are "spam" and "ham", we will use LabelEncoder to encode these values to integers.

Then we try a few models.

In [ ]:
y_train = train_data["Type"]
encoder = LabelEncoder()
type_encoded = encoder.fit_transform(y_train)

### Logistic Regression

In [ ]:
log_clf = LogisticRegression(solver="liblinear", random_state=42)
score = cross_val_score(log_clf, train_prepared, type_encoded, cv=3, verbose=3)
score.mean()

 ### SGD

In [ ]:
sgd_clf = SGDClassifier(random_state=42)
score = cross_val_score(sgd_clf, train_prepared, type_encoded, cv=3, verbose=3)
score.mean()

### MultinomialNB

In [ ]:
mnb_clf = MultinomialNB()
score = cross_val_score(mnb_clf, train_prepared, type_encoded, cv=3, verbose=3)
score.mean()

At this step, we use the test data and analyze our recall and precision.

### BernoulliNB

In [ ]:
bnb_clf = BernoulliNB()
score = cross_val_score(bnb_clf, train_prepared, type_encoded, cv=3, verbose=3)
score.mean()

In [ ]:
from sklearn.metrics import precision_score, recall_score

y_test = test_data["Type"]
encoder = LabelEncoder()
type_encoded = encoder.fit_transform(y_test)

X_test_transformed = full_pipeline.transform(test_data)

bnb_clf = BernoulliNB()
bnb_clf.fit(train_prepared, y_train)

y_pred = bnb_clf.predict(X_test_transformed)
encoder = LabelEncoder()
y_pred = encoder.fit_transform(y_pred)

print("Precision BernoulliNB: {:.2f}%".format(100 * precision_score(type_encoded, y_pred)))
print("Recall BernoulliNB: {:.2f}%\n".format(100 * recall_score(type_encoded, y_pred)))

mnb_clf = MultinomialNB()
mnb_clf.fit(train_prepared, y_train)

y_pred = mnb_clf.predict(X_test_transformed)
encoder = LabelEncoder()
y_pred = encoder.fit_transform(y_pred)

print("Precision MultinomialNB: {:.2f}%".format(100 * precision_score(type_encoded, y_pred)))
print("Recall MultinomialNB: {:.2f}%\n".format(100 * recall_score(type_encoded, y_pred)))

sgd_clf = SGDClassifier(random_state=42)
sgd_clf.fit(train_prepared, y_train)

y_pred = sgd_clf.predict(X_test_transformed)
encoder = LabelEncoder()
y_pred = encoder.fit_transform(y_pred)

print("Precision SGD: {:.2f}%".format(100 * precision_score(type_encoded, y_pred)))
print("Recall SGD: {:.2f}%\n".format(100 * recall_score(type_encoded, y_pred)))

log_clf = LogisticRegression(solver="liblinear", random_state=42)
log_clf.fit(train_prepared, y_train)

y_pred = log_clf.predict(X_test_transformed)
encoder = LabelEncoder()
y_pred = encoder.fit_transform(y_pred)

print("Precision Logistic Regression: {:.2f}%".format(100 * precision_score(type_encoded, y_pred)))
print("Recall Logistic Regression: {:.2f}%".format(100 * recall_score(type_encoded, y_pred)))